In [1]:
# 3rd party # remove imports that are not needed
import numpy as np
import pandas as pd
import pylab as plt
import h5py
import matplotlib
%matplotlib inline
from astropy.utils.data import download_file  #import file from URL
from matplotlib.ticker import NullFormatter
import astropy.table
import subprocess as sp
import glob
import healpy as hp
#/data/des90.a/data/raulteix/data/phz_6416_no_wise.fits, /data/des90.a/data/raulteix/data/DELVE_w_photozs_06416.fits, /data/des90.a/data/raulteix/data/phz_6416_no_wise.fits
#data = astropy.table.Table.read('/data/des90.a/data/raulteix/data/phz_09602_no_wise.fits')

In [2]:
flist = glob.glob('/data/des91.b/data/kadrlica/projects/delve/cat/y3t1/v1/cat/cat*.fits')
pxnums = []
for fname in flist:
    pxnums.append(int(fname.split('.')[1].split('_')[-1]))
pxnums = np.array(pxnums)
ra, dec = hp.pix2ang(32, pxnums, lonlat=True)

data_boss_north = astropy.table.Table.read('/data/des90.a/data/raulteix/data/speczs/galaxy_DR12v5_CMASSLOWZTOT_North.fits.gz')
data_boss_south = astropy.table.Table.read('/data/des90.a/data/raulteix/data/speczs/galaxy_DR12v5_CMASSLOWZTOT_South.fits.gz')
data_boss_all = astropy.table.vstack([data_boss_north, data_boss_south])
pxnums_boss = hp.ang2pix(32, data_boss_all['RA'], data_boss_all['DEC'], lonlat=True)
overlap_boss_delve = np.in1d(pxnums_boss, pxnums)

data_boss_good = data_boss_all[overlap_boss_delve]

px_overlap = np.unique(pxnums_boss[overlap_boss_delve])

In [3]:
pxlist = np.char.add(np.zeros_like(np.array(list(px_overlap))).astype('str'), np.array(list(px_overlap)).astype('str'))

In [4]:
import sys
np.set_printoptions(threshold=sys.maxsize)
np.array([list(RAdelve), list(DECdelve)]).T.shape

NameError: name 'RAdelve' is not defined

In [5]:
RAdelve, DECdelve = hp.pix2ang(32, px_overlap, lonlat=True)

In [6]:
mask1 = np.array(RAdelve)<180
mask2 = np.array(RAdelve)>120
mask3 = np.array(RAdelve)>-180
mask4 = np.array(RAdelve)<-120

mask5 = np.array(DECdelve)>-10
mask6 = np.array(DECdelve)<10
mask12 = mask1*mask2
mask34 = mask3*mask4
mask56 = mask5*mask6
mask1234 = (mask12+mask34)
mask = mask1234 * mask56

In [7]:
px_overlap[mask]

5099
5100
5101
5102
5103
5104
5105
5106
5107
5108
5109


In [9]:
RA_DECtable = astropy.table.hstack([RAdelve[mask], DECdelve[mask]])

In [10]:
RA_DECtable['DEC_1'].name = 'RA'
RA_DECtable['DEC_2'].name = 'DEC'

In [33]:
hp.ang2pix(32, 122.34374999999999, 9.594068226860458, lonlat=True)

5099

In [9]:
hp.ang2pix(32, 2.8125, -1.1937484371410818, lonlat=True)

6209

In [34]:
pixel = '05099'

In [35]:
list_lines = ['COLUMNS     /data/des90.a/data/raulteix/software/DESC_BPZ/tests/DELVEdata_example.columns\n', f'OUTPUT\t    /data/des90.a/data/raulteix/software/DESC_BPZ/tests/DELVEdata_example_output_{pixel}_no_wise.bpz.h5\n', 'SPECTRA     CWWSB4.list\n', 'PRIOR\t    hdfn_gen\n', 'DZ          0.01\n', 'ZMIN        0.005\n', 'ZMAX        3.505\n', 'MAG         yes\n', 'NEW_AB      no\n', 'MADAU\t    no #TURN OFF MADAU!!!!\n', 'EXCLUDE     none\n', 'CHECK       yes\n', '#ZC          1.0,2.0\n', '#FC          0.2,0.4\n', 'VERBOSE     no\n', '#INTERP      0\n', 'ODDS        0.68\n', 'PROBS       no\n', 'PROBS_LITE  DELVEdata_09602_probs.h5\n', 'GET_Z       yes\n', 'INTERACTIVE yes\n', 'PLOTS       no\n', '#ONLY_TYPE   yes\n']
pars = open('/data/des90.a/data/raulteix/software/DESC_BPZ/tests/DELVEdata_example.pars', mode='w')
pars.writelines(list_lines)
pars.close()

#tab_clecio = astropy.table.Table.read(f'/data/des91.b/data/debom/delve/photoz/dr1/v0/cat_hpx_{pixel}_phz.fits')
tab_alex = astropy.table.Table.read(f'/data/des91.b/data/kadrlica/projects/delve/cat/y3t1/v1/cat/cat_hpx_{pixel}.fits')
#tab_all = astropy.table.join(tab_clecio, tab_alex)

bandlist = ['G', 'R', 'I', 'Z']
for band in bandlist:
    tab_alex[f'EXTENDED_CLASS_{band}'] = 1*((tab_alex[f'SPREAD_MODEL_{band}'] + 3*tab_alex[f'SPREADERR_MODEL_{band}']) > 0.005) + 1*((tab_alex[f'SPREAD_MODEL_{band}'] + tab_alex[f'SPREADERR_MODEL_{band}']) > 0.003) + 1*((tab_alex[f'SPREAD_MODEL_{band}'] - tab_alex[f'SPREADERR_MODEL_{band}']) > 0.003)

tab_alex_f = tab_alex[tab_alex['EXTENDED_CLASS_G']>1]

runthroughbpz = tab_alex_f['QUICK_OBJECT_ID', 'MAG_AUTO_G', 'MAG_AUTO_R', 'MAG_AUTO_I', 'MAG_AUTO_Z', 'MAGERR_AUTO_G', 'MAGERR_AUTO_R', 'MAGERR_AUTO_I', 'MAGERR_AUTO_Z'] #[tab_all['extended_class']>1]

In [ ]:
runthroughbpz

In [57]:
for label in runthroughbpz.columns:
    print(np.var(runthroughbpz[label]), label)

3174242335.8576527 QUICK_OBJECT_ID
1.1640885 MAG_AUTO_G
575.81323 MAG_AUTO_R
925.2146 MAG_AUTO_I
1295.4943 MAG_AUTO_Z
0.13248432 MAGERR_AUTO_G
945.3097 MAGERR_AUTO_R
1497.7654 MAGERR_AUTO_I
2072.365 MAGERR_AUTO_Z


In [58]:
pdbpz = pd.DataFrame(data=runthroughbpz['QUICK_OBJECT_ID'], columns = ['QUICK_OBJECT_ID'])
for label in runthroughbpz.columns:
    pdbpz[label] = runthroughbpz[label]
    
pdbpz.to_hdf(f'/data/des90.a/data/raulteix/software/DESC_BPZ/tests/DELVEdata_{pixel}_no_wise.h5', key='df')

In [61]:
catalog_name = f'/data/des90.a/data/raulteix/software/DESC_BPZ/tests/DELVEdata_{pixel}_no_wise.h5'
sp.run(['python', '/data/des90.a/data/raulteix/software/DESC_BPZ/scripts/bpz.py', catalog_name, '-P', '/data/des90.a/data/raulteix/software/DESC_BPZ/tests/DELVEdata_example.pars'])

CompletedProcess(args=['python', '/data/des90.a/data/raulteix/software/DESC_BPZ/scripts/bpz.py', '/data/des90.a/data/raulteix/software/DESC_BPZ/tests/DELVEdata_05099_no_wise.h5', '-P', '/data/des90.a/data/raulteix/software/DESC_BPZ/tests/DELVEdata_example.pars'], returncode=0)

In [64]:
bpzdata = h5py.File(f'/data/des90.a/data/raulteix/software/DESC_BPZ/tests/DELVEdata_example_output_{pixel}_no_wise.bpz.h5')

In [65]:
bpzdatapd = pd.DataFrame(data=bpzdata['ID'], columns =['QUICK_OBJECT_ID'])

In [66]:
for key in list(bpzdata.keys()):
    bpzdatapd[key]=list(bpzdata[key])

In [67]:
bpzdatatable = astropy.table.Table.from_pandas(bpzdatapd)

In [68]:
data = astropy.table.join(bpzdatatable, tab_alex_f)

In [ ]:
bpzdatapd

In [ ]:
data

In [69]:
def compute_median_bias(zpred, ztrue):
    """Compute the median bias                                                                                                                                                                              
                                                                                                                                                                                                            
    Parameters                                                                                                                                                                                              
    ----------                                                                                                                                                                                              
    zpred : predicted redshift                                                                                                                                                                              
    ztest : spectroscopic redshift                                                                                                                                                                          
                                                                                                                                                                                                            
    Returns                                                                                                                                                                                                 
    -------                                                                                                                                                                                                 
    bias : the bias (ztrue - zpred)                                                                                                                                                                         
    """
    return np.median(ztrue - zpred)

def compute_sigma68(zpred, ztrue):
    """Compute the rms error between zpred and ztrue                                                                                                                                                        
                                                                                                                                                                                                            
    Parameters                                                                                                                                                                                              
    ----------                                                                                                                                                                                              
    zpred : predicted redshift                                                                                                                                                                              
    ztrue : spectroscopic redshift                                                                                                                                                                          
                                                                                                                                                                                                            
    Returns                                                                                                                                                                                                 
    -------                                                                                                                                                                                                 
    rms_error : root-mean square error                                                                                                                                                                      
    """
    # This question asks for the rms error of the difference                                                                                                                                                
    # distribution.                                                                                                                                                                                         
    return np.std(ztrue - zpred)

    # This definition of sigma68 differs from that in Sanchez et. al                                                                                                                                        
    # 2015, which defined sigma_68 to be the 68% containment about the                                                                                                                                      
    # median.  Sanchez's value would be something more like:                                                                                                                                                
    #return np.abs(np.diff(np.percentile(ztrue-zpred,[16,84])))/2.                                                                                                                                          
    # which gives a significantly smaller value.                                                                                                                                                            

def compute_outlier_fraction(zpred, ztrue, nsigma=2):
    """Compute the outlier fraction between zpred and ztrue                                                                                                                                                 
                                                                                                                                                                                                            
    Parameters                                                                                                                                                                                              
    ----------                                                                                                                                                                                              
    zpred : predicted redshift                                                                                                                                                                              
    ztrue : spectroscopic redshift                                                                                                                                                                          
    nsigma: number of sigma to classify outliers                                                                                                                                                            
                                                                                                                                                                                                            
    Returns                                                                                                                                                                                                 
    -------                                                                                                                                                                                                 
    frac : outlier fraction                                                                                                                                                                                 
    """
    sigma68 = compute_sigma68(zpred,ztrue)
    return np.sum(abs(zpred - ztrue)>nsigma*sigma68)/float(len(zpred))

In [ ]:
data['EXTENDED_CLASS_G'] = ((data['SPREAD_MODEL_G'] + 3*data['SPREADERR_MODEL_G']) > 0.005) + ((data['SPREAD_MODEL_G'] + data['SPREADERR_MODEL_G']) > 0.003) + ((data['SPREAD_MODEL_G'] - data['SPREADERR_MODEL_G']) > 0.003)

In [ ]:
data = data[data['EXTENDED_CLASS_G']>1]

In [70]:
z_nn, z_bpz = data['z'], data['Z_B']

KeyError: 'z'

In [ ]:
plt.hist(z_bpz-z_nn, range=(-1.2, 1.2), bins = 'auto')
plt.xlabel('bpz best redshift - ML redshift')
plt.show()
plt.scatter(z_bpz, z_bpz-z_nn)
plt.xlabel('bpz best redshift')
plt.ylabel('bpz best redshift - ML redshift')
plt.ylim(2, -2)
plt.show()

In [ ]:
n, bins, patches = plt.hist(z_bpz, range = (0.01, 1.2), bins = 22)
plt.xlabel('bpz best redshift')

In [ ]:
nc, binsc, patchesc = plt.hist(z_nn, range = (0.01, 1.2), bins = 22)

In [ ]:
plt.hist2d(z_bpz, z_bpz-z_nn, range=[(0, 1.2),(-1.2, 1.2)], bins=30)
plt.axhline(0.0, ls='--', c='r')
plt.xlabel('bpz best redshift')
plt.ylabel('bpz best redshift - ML redshift')
plt.show()

In [ ]:
plt.hist2d(z_bpz, z_nn, range=[(0, 1.2),(0, 1.2)], bins=30)
plt.plot(np.linspace(0, 1.2), np.linspace(0, 1.2), 'r--')
plt.ylabel('ML redshift')
plt.xlabel('bpz best redshift')
plt.show()

In [ ]:
z_spec = data['zspec']

In [ ]:
plt.hist2d(z_nn, z_spec, range=[(0, 1.2),(0, 1.2)], bins=30)
plt.plot(np.linspace(0, 1.2), np.linspace(0, 1.2), 'r--')
plt.xlabel('ML redshift')
plt.ylabel('spectroscopic redshift')
plt.show()

In [ ]:
plt.hist2d(z_bpz, z_spec, range=[(0, 1.2),(0, 1.2)], bins=35)
plt.plot(np.linspace(0, 1.2), np.linspace(0, 1.2), 'r--')
plt.xlabel('bpz best redshift')
plt.ylabel('spectroscopic redshift')
plt.show()

## Bias

In [ ]:
z_err_nn = data['zerr']

**BPZ BIAS**

In [ ]:
bias_bpz = z_bpz - z_spec
bias_clecio = z_nn - z_spec
std_bpz = np.std(z_bpz, ddof = 1)

In [ ]:
n, bins, patches = plt.hist(bias_bpz, range=(0, 1.2), bins = 'auto')
plt.xlabel('bias bpz relative to spec z')

In [ ]:
z_err_bpz_68 = compute_sigma68(z_bpz, z_spec)

In [ ]:
z_err_bpz_68

In [ ]:
plt.scatter(z_bpz, bias_bpz/z_err_bpz_68)
plt.xlabel('bpz best redshift')
plt.ylabel('bias')
plt.ylim(-1.2, 1.2)

In [ ]:
bias_bpz_med = np.median(bias_bpz)
print('median bias =', bias_bpz_med)

## Outliers

In [ ]:
compute_outlier_fraction(z_bpz, z_nn, nsigma = 3) #6.817977118638658406979402570869)

In [ ]:
len(bias_bpz[bias_bpz<-6])/len(bias_bpz)

In [ ]:
nsigma = 3
sigma68 = np.std(bias_bpz[bias_bpz>-6])
np.sum(abs(bias_bpz[bias_bpz>-6])>nsigma*sigma68)/float(len(z_spec[bias_bpz>-6]))

In [ ]:
mask1 = z_spec != 99
mask2 = z_spec != -99
mask = mask1*mask2

In [ ]:
compute_outlier_fraction(z_bpz[mask], z_spec[mask], nsigma = 2)

In [ ]:
compute_outlier_fraction(z_nn, z_spec, nsigma = 3)